In [1]:
!pip install kaggle

  Running setup.py bdist_wheel for kaggle ... done
  Stored in directory: /Users/quentin/Library/Caches/pip/wheels/61/d6/e6/38fc0e2316f49b91aafdaf9a8d7756a9906b2ad7e501c1e165
Successfully built kaggle
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [175]:
from pandas import read_csv

In [176]:
train_data = read_csv('./data/train.csv')
test_data = read_csv('./data/test.csv')

In [177]:
train_data = train_data.set_index('ID')

In [178]:
# Removing columns with only 0
train_data = train_data.loc[:, (train_data != 0).any(axis=0)]

In [179]:
train_data.shape

(4459, 4736)

In [172]:
import numpy as np

In [173]:
import pandas as pd

In [174]:
from sklearn.model_selection import train_test_split

In [181]:
train_data.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [182]:
train_x = train_data.drop(["target"], axis=1)
train_y = np.log1p(train_data["target"].values)

In [183]:
x_dev, x_val, y_dev, y_val = train_test_split(train_x, train_y, test_size=0.2) # consider random_state

In [184]:
import lightgbm as lgb

In [185]:
def run_lgbm(x_train, y_train, x_val, y_val, x_test):
    params = {
"objective": "regression",
        "metric": "rmse",
        "num_leaves": 40,
        "learning_rate": 0.005,
        "bagging_fraction": 0.7,
        "feature_fraction": 0.5,
        "bagging_frequency": 5,
        "bagging_seed": 42,
        "verbosity": -1,
        "seed": 42,    
    }
    lgtrain = lgb.Dataset(x_train, label=y_train)
    lgvalid = lgb.Dataset(x_val, label=y_val)
    evals_result = {}
    model = lgb.train(params, lgtrain, 3000,
                      valid_sets=[lgtrain, lgvalid],
                      evals_result = evals_result,
                      early_stopping_rounds=100)
    
    y_hat_test = np.expm1(model.predict(x_test, num_iteration=model.best_iteration))
    return y_hat_test, model, evals_result

In [186]:
y_hat_test_lgb, model, eval_results = run_lgbm(x_dev, y_dev, x_val, y_val, train_x)

[1]	training's rmse: 1.74304	valid_1's rmse: 1.76829
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1.74024	valid_1's rmse: 1.76623
[3]	training's rmse: 1.73748	valid_1's rmse: 1.76405
[4]	training's rmse: 1.73478	valid_1's rmse: 1.7619
[5]	training's rmse: 1.73208	valid_1's rmse: 1.75973
[6]	training's rmse: 1.72954	valid_1's rmse: 1.7578
[7]	training's rmse: 1.72702	valid_1's rmse: 1.75562
[8]	training's rmse: 1.72462	valid_1's rmse: 1.75361
[9]	training's rmse: 1.72213	valid_1's rmse: 1.75167
[10]	training's rmse: 1.71946	valid_1's rmse: 1.74957
[11]	training's rmse: 1.71675	valid_1's rmse: 1.74753
[12]	training's rmse: 1.71418	valid_1's rmse: 1.74562
[13]	training's rmse: 1.71165	valid_1's rmse: 1.74373
[14]	training's rmse: 1.70926	valid_1's rmse: 1.74187
[15]	training's rmse: 1.7067	valid_1's rmse: 1.73994
[16]	training's rmse: 1.70436	valid_1's rmse: 1.73817
[17]	training's rmse: 1.70178	valid_1's rmse: 1.73631
[18]	training's rmse: 1.69924	v

[152]	training's rmse: 1.44913	valid_1's rmse: 1.56519
[153]	training's rmse: 1.44768	valid_1's rmse: 1.56448
[154]	training's rmse: 1.4463	valid_1's rmse: 1.5636
[155]	training's rmse: 1.44496	valid_1's rmse: 1.56288
[156]	training's rmse: 1.4436	valid_1's rmse: 1.56192
[157]	training's rmse: 1.44217	valid_1's rmse: 1.561
[158]	training's rmse: 1.44083	valid_1's rmse: 1.56012
[159]	training's rmse: 1.43934	valid_1's rmse: 1.55921
[160]	training's rmse: 1.43803	valid_1's rmse: 1.55837
[161]	training's rmse: 1.43671	valid_1's rmse: 1.55754
[162]	training's rmse: 1.43529	valid_1's rmse: 1.55671
[163]	training's rmse: 1.43397	valid_1's rmse: 1.5559
[164]	training's rmse: 1.43261	valid_1's rmse: 1.5552
[165]	training's rmse: 1.43124	valid_1's rmse: 1.55436
[166]	training's rmse: 1.4299	valid_1's rmse: 1.55365
[167]	training's rmse: 1.42855	valid_1's rmse: 1.55257
[168]	training's rmse: 1.42721	valid_1's rmse: 1.5519
[169]	training's rmse: 1.42585	valid_1's rmse: 1.55128
[170]	training's rm

[307]	training's rmse: 1.27463	valid_1's rmse: 1.47796
[308]	training's rmse: 1.27373	valid_1's rmse: 1.47773
[309]	training's rmse: 1.27288	valid_1's rmse: 1.47745
[310]	training's rmse: 1.272	valid_1's rmse: 1.47695
[311]	training's rmse: 1.27103	valid_1's rmse: 1.47647
[312]	training's rmse: 1.27016	valid_1's rmse: 1.47622
[313]	training's rmse: 1.26931	valid_1's rmse: 1.47591
[314]	training's rmse: 1.26844	valid_1's rmse: 1.47556
[315]	training's rmse: 1.26754	valid_1's rmse: 1.4753
[316]	training's rmse: 1.26666	valid_1's rmse: 1.47506
[317]	training's rmse: 1.2658	valid_1's rmse: 1.47465
[318]	training's rmse: 1.26492	valid_1's rmse: 1.47434
[319]	training's rmse: 1.26406	valid_1's rmse: 1.47408
[320]	training's rmse: 1.26318	valid_1's rmse: 1.47373
[321]	training's rmse: 1.26236	valid_1's rmse: 1.47319
[322]	training's rmse: 1.26154	valid_1's rmse: 1.47285
[323]	training's rmse: 1.26066	valid_1's rmse: 1.47253
[324]	training's rmse: 1.25975	valid_1's rmse: 1.47218
[325]	training

[460]	training's rmse: 1.15922	valid_1's rmse: 1.44326
[461]	training's rmse: 1.1586	valid_1's rmse: 1.44303
[462]	training's rmse: 1.15796	valid_1's rmse: 1.443
[463]	training's rmse: 1.15734	valid_1's rmse: 1.44285
[464]	training's rmse: 1.15669	valid_1's rmse: 1.44267
[465]	training's rmse: 1.15603	valid_1's rmse: 1.44246
[466]	training's rmse: 1.15541	valid_1's rmse: 1.44229
[467]	training's rmse: 1.15476	valid_1's rmse: 1.44223
[468]	training's rmse: 1.15415	valid_1's rmse: 1.44206
[469]	training's rmse: 1.1535	valid_1's rmse: 1.44189
[470]	training's rmse: 1.15285	valid_1's rmse: 1.44184
[471]	training's rmse: 1.15217	valid_1's rmse: 1.44172
[472]	training's rmse: 1.15154	valid_1's rmse: 1.44152
[473]	training's rmse: 1.15088	valid_1's rmse: 1.44138
[474]	training's rmse: 1.15026	valid_1's rmse: 1.44133
[475]	training's rmse: 1.14965	valid_1's rmse: 1.44116
[476]	training's rmse: 1.14901	valid_1's rmse: 1.44108
[477]	training's rmse: 1.14841	valid_1's rmse: 1.44095
[478]	training

[612]	training's rmse: 1.07521	valid_1's rmse: 1.4308
[613]	training's rmse: 1.07475	valid_1's rmse: 1.43077
[614]	training's rmse: 1.0743	valid_1's rmse: 1.43073
[615]	training's rmse: 1.07388	valid_1's rmse: 1.43075
[616]	training's rmse: 1.07339	valid_1's rmse: 1.43074
[617]	training's rmse: 1.07289	valid_1's rmse: 1.43068
[618]	training's rmse: 1.07239	valid_1's rmse: 1.43058
[619]	training's rmse: 1.07192	valid_1's rmse: 1.43058
[620]	training's rmse: 1.07143	valid_1's rmse: 1.43058
[621]	training's rmse: 1.07096	valid_1's rmse: 1.43057
[622]	training's rmse: 1.07051	valid_1's rmse: 1.43062
[623]	training's rmse: 1.07005	valid_1's rmse: 1.43059
[624]	training's rmse: 1.0696	valid_1's rmse: 1.43051
[625]	training's rmse: 1.06914	valid_1's rmse: 1.43042
[626]	training's rmse: 1.06864	valid_1's rmse: 1.43042
[627]	training's rmse: 1.06815	valid_1's rmse: 1.43038
[628]	training's rmse: 1.06765	valid_1's rmse: 1.43029
[629]	training's rmse: 1.06722	valid_1's rmse: 1.43029
[630]	trainin

[765]	training's rmse: 1.01125	valid_1's rmse: 1.42876
[766]	training's rmse: 1.01087	valid_1's rmse: 1.42864
[767]	training's rmse: 1.01048	valid_1's rmse: 1.4286
[768]	training's rmse: 1.01009	valid_1's rmse: 1.42864
[769]	training's rmse: 1.00971	valid_1's rmse: 1.4287
[770]	training's rmse: 1.00935	valid_1's rmse: 1.42876
[771]	training's rmse: 1.00899	valid_1's rmse: 1.42877
[772]	training's rmse: 1.00864	valid_1's rmse: 1.42885
[773]	training's rmse: 1.00823	valid_1's rmse: 1.42894
[774]	training's rmse: 1.00785	valid_1's rmse: 1.42899
[775]	training's rmse: 1.00747	valid_1's rmse: 1.42896
[776]	training's rmse: 1.00711	valid_1's rmse: 1.42896
[777]	training's rmse: 1.00676	valid_1's rmse: 1.42896
[778]	training's rmse: 1.00636	valid_1's rmse: 1.42888
[779]	training's rmse: 1.00603	valid_1's rmse: 1.42891
[780]	training's rmse: 1.00566	valid_1's rmse: 1.4289
[781]	training's rmse: 1.00529	valid_1's rmse: 1.42891
[782]	training's rmse: 1.00495	valid_1's rmse: 1.42893
[783]	trainin

In [187]:
import xgboost as xgb

In [188]:
def run_xgb(x_train, y_train, x_val, y_val, x_test):
    params = {'objective': 'reg:linear', 
      'eval_metric': 'rmse',
      'eta': 0.001,
      'max_depth': 10, 
      'subsample': 0.6, 
      'colsample_bytree': 0.6,
      'alpha':0.001,
      'random_state': 42, 
      'silent': True}
    
    xgtrain = xgb.DMatrix(x_train, y_train)
    xgvalid = xgb.DMatrix(x_val, y_val)
    
    eval_results = [(xgtrain, 'train'), (xgvalid, 'valid')]
    
    model = xgb.train(params, xgtrain, 5000, eval_results, 
                      maximize=False, 
                      verbose_eval=100, 
                      early_stopping_rounds=100)
    
    xgtest_x = xgb.DMatrix(x_test)
    x_hat_test = np.expm1(model.predict(xgtest_x, ntree_limit=model.best_ntree_limit))
    return x_hat_test, model, eval_results

In [189]:
y_hat_test_xgb, model, eval_results = run_xgb(x_dev, y_dev, x_val, y_val, train_x)

[0]	train-rmse:14.1028	valid-rmse:14.0157
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[100]	train-rmse:12.7817	valid-rmse:12.6954
[200]	train-rmse:11.5883	valid-rmse:11.5035
[300]	train-rmse:10.5103	valid-rmse:10.4275
[400]	train-rmse:9.53694	valid-rmse:9.45707
[500]	train-rmse:8.65829	valid-rmse:8.58281
[600]	train-rmse:7.8655	valid-rmse:7.79527
[700]	train-rmse:7.14982	valid-rmse:7.0865
[800]	train-rmse:6.5038	valid-rmse:6.44764
[900]	train-rmse:5.9212	valid-rmse:5.87287
[1000]	train-rmse:5.39604	valid-rmse:5.35671
[1100]	train-rmse:4.92244	valid-rmse:4.89257
[1200]	train-rmse:4.49581	valid-rmse:4.47696
[1300]	train-rmse:4.11186	valid-rmse:4.10404
[1400]	train-rmse:3.76639	valid-rmse:3.77037
[1500]	train-rmse:3.45583	valid-rmse:3.47319
[1600]	train-rmse:3.17746	valid-rmse:3.20859
[1700]	train-rmse:2.92733	valid-rmse:2.97308
[1800]	train-rmse:2.70339	valid-rmse:2.76458
[1900]	train-rm

In [190]:
submission = pd.read_csv('../input/sample_submission.csv')

submission_lgb = pd.DataFrame()
submission_lgb["target"] = y_hat_test_lgb

submission_xgb = pd.DataFrame()
submission_xgb["target"] = y_hat_test_xgb

submission["target"] = (sub_lgb["target"] * 0.5 + sub_xgb["target"] * 0.5)

FileNotFoundError: File b'../input/sample_submission.csv' does not exist